In [1]:
import sys  
sys.path.insert(0, "helpers/")
from aux_functions import (dateindex_from_timestamp, dateindex_from_weeknum)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/Users/Geri/.local/share/virtualenvs/thesis-ez8c3QqQ/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
all_daily_sales = pd.read_csv("../../datasets/dati_nunalie/vendite_giornaliere.csv")
all_daily_sales.head()

,Unnamed: 0,qty,codice esterno,data,tab negozio
0,0,1,LQG16F54JE-16,2016-09-25 15:33:04,CREMO
1,1,1,ALMA2470FA-19,2019-12-15 14:51:34,NICHE
2,2,1,VE119SE-16,2016-07-30 13:31:31,SASA2
3,3,1,RR4NE-15,2016-02-21 13:14:47,LIDOV
4,4,1,829929CBL-17,2017-07-31 13:54:17,BA_02


In [3]:
all_daily_sales["tab negozio"].unique()

array(['CREMO', 'NICHE', 'SASA2', 'LIDOV', 'BA_02', 'BARI1', 'TIBUR',
       'BIELA', 'MODEN', 'APRIL', 'NUORO', 'PASTR', 'EURR2', 'CATA1',
       'LOSAN', 'LECCE', 'MOLFE', 'BELIN', 'SANRE', 'ALGHE', 'TSGIU',
       'TRIES', 'VILES', 'CREMA', 'ALBIG', 'MESTR', 'CHLUG', 'OLBIA',
       'ARESE', 'RONCA', 'TUSC2', 'PRADA', 'IMOLA', 'BOLZA', 'VERON',
       'TRENT', 'BRAGE', 'AOSTA', 'PONTE', 'SINAI', 'LEIDA', 'RMNAZ',
       'FIUM1', 'FRIBU', 'BRESA', 'BASAN', 'CAPAO', 'MERAN', 'IGLES',
       'CALIA', 'VALVI', 'LUCCA', 'JUVE', 'CARBO', 'TORIN', 'CHIAV',
       'NOVRA', 'BUSNA', 'SESTU', 'SASAR', 'GENOA', 'GEMON', 'ROVIO',
       'ORIST', 'RAVEN', 'LIVOR', 'SVI01', 'TREME', 'VENEZ', 'SION',
       'MESSI', 'KATAN', 'TUSC1', 'REMIL', 'VITER', 'BEINA', 'GTOCE',
       'FERAR', 'PIAVE', 'MASSA', 'CONEG', 'PALE2', 'CAPDO', 'OZIER',
       'TREVI', 'PAVIA', 'VETRA', 'REGIO', 'BOSCO', 'GROSS', 'MAREN',
       'SIDER', 'SAVON', 'BRUNI', 'PAD01', 'PIACE', 'MEGLI', 'PAESE',
       'PORTE', 'PABEL

In [4]:
def create_daily_store_sales_ts(all_daily_data, wanted_store):
    # Select data only from the wanted store loc
    localized_data = all_daily_data[all_daily_data["tab negozio"] == wanted_store]
    
    # Drop unnamed column and listino column, we won't be using those
    localized_data = localized_data.drop(["Unnamed: 0", "tab negozio"], axis=1)

    # Build datetime index
    localized_data["time"] = [dateindex_from_timestamp(timestamp[:10]) for timestamp in localized_data["data"].values]
    localized_data = localized_data.set_index(["time"])
    localized_data.index = pd.to_datetime(localized_data.index) # make ts

    # Sort the data based on the new time index
    localized_data.sort_index(inplace=True)

    localized_data = localized_data.drop(["data", "codice esterno"], axis=1)

    # Aggregate the data to get the total sales of each day in case of duplicates
    localized_data_daily = localized_data.groupby(pd.Grouper(freq = "D")).agg({
        "qty": np.sum
    })
    
    return localized_data_daily

In [5]:
def create_weekly_store_sales_ts(all_daily_data, wanted_store):
    # Select data only from the wanted store loc
    localized_data = all_daily_data[all_daily_data["tab negozio"] == wanted_store]
    
    # Drop unnamed column and listino column, we won't be using those
    localized_data = localized_data.drop(["Unnamed: 0", "tab negozio"], axis=1)

    # Build datetime index
    localized_data["time"] = [dateindex_from_timestamp(timestamp[:10]) for timestamp in localized_data["data"].values]
    localized_data = localized_data.set_index(["time"])
    localized_data.index = pd.to_datetime(localized_data.index) # make ts

    # Sort the data based on the new time index
    localized_data.sort_index(inplace=True)

    localized_data = localized_data.drop(["data", "codice esterno"], axis=1)

    # Aggregate the data to get the total sales of each day in case of duplicates
    localized_data_weekly = localized_data.groupby(pd.Grouper(freq = "W-MON")).agg({
        "qty": np.sum
    })
    
    # We won't be using 2020 data
    localized_data_weekly.drop(localized_data_weekly.loc["2020"].index, inplace=True)

    
    return localized_data_weekly

In [6]:
# Milan
milan_cc_arese_data = create_daily_store_sales_ts(all_daily_sales, "ARESE")
milan_cc_busna_data = create_daily_store_sales_ts(all_daily_sales, "BUSNA")

# Turin
turin_cc_niche_data = create_daily_store_sales_ts(all_daily_sales, "NICHE")
turin_cc_beina_data = create_daily_store_sales_ts(all_daily_sales, "BEINA")
turin_cc_torin_data = create_daily_store_sales_ts(all_daily_sales, "TORIN")
turin_cc_todor_data = create_daily_store_sales_ts(all_daily_sales, "TODOR")
turin_cc_juve_data = create_daily_store_sales_ts(all_daily_sales, "JUVE")

# Rome
rome_cc_eur2_data = create_daily_store_sales_ts(all_daily_sales, "EURR2")
rome_neg_rmnaz_data = create_daily_store_sales_ts(all_daily_sales, "RMNAZ")
rome_neg_tusc1_data = create_daily_store_sales_ts(all_daily_sales, "TUSC1")
rome_neg_tusc2_data = create_daily_store_sales_ts(all_daily_sales, "TUSC2")
rome_neg_tibur_data = create_daily_store_sales_ts(all_daily_sales, "TIBUR")

In [7]:
# Milan
milan_cc_arese_data_w = create_weekly_store_sales_ts(all_daily_sales, "ARESE")
milan_cc_busna_data_w = create_weekly_store_sales_ts(all_daily_sales, "BUSNA")

# Turin
turin_cc_niche_data_w = create_weekly_store_sales_ts(all_daily_sales, "NICHE")
turin_cc_beina_data_w = create_weekly_store_sales_ts(all_daily_sales, "BEINA")
turin_cc_torin_data_w = create_weekly_store_sales_ts(all_daily_sales, "TORIN")
turin_cc_todor_data_w = create_weekly_store_sales_ts(all_daily_sales, "TODOR")
turin_cc_juve_data_w = create_weekly_store_sales_ts(all_daily_sales, "JUVE")

# Rome
rome_cc_eur2_data_w = create_weekly_store_sales_ts(all_daily_sales, "EURR2")
rome_neg_rmnaz_data_w = create_weekly_store_sales_ts(all_daily_sales, "RMNAZ")
rome_neg_tusc1_data_w = create_weekly_store_sales_ts(all_daily_sales, "TUSC1")
rome_neg_tusc2_data_w = create_weekly_store_sales_ts(all_daily_sales, "TUSC2")
rome_neg_tibur_data_w = create_weekly_store_sales_ts(all_daily_sales, "TIBUR")

In [8]:
turin_cc_niche_data_w

,qty
time,
2017-10-30,530
2017-11-06,552
2017-11-13,357
2017-11-20,338
2017-11-27,619
...,...
2019-12-02,879
2019-12-09,322
2019-12-16,485


In [9]:
# Export daily
milan_cc_arese_data.to_csv("milan_cc_arese_data.csv")
milan_cc_busna_data.to_csv("milan_cc_busna_data.csv")
turin_cc_niche_data.to_csv("turin_cc_niche_data.csv")
turin_cc_beina_data.to_csv("turin_cc_beina_data.csv")
turin_cc_torin_data.to_csv("turin_cc_torin_data.csv")
turin_cc_todor_data.to_csv("turin_cc_todor_data.csv")
turin_cc_juve_data.to_csv("turin_cc_juve_data.csv")
rome_cc_eur2_data.to_csv("rome_cc_eur2_data.csv")
rome_neg_rmnaz_data.to_csv("rome_neg_rmnaz_data.csv")
rome_neg_tusc1_data.to_csv("rome_neg_tusc1_data.csv")
rome_neg_tusc2_data.to_csv("rome_neg_tusc2_data.csv")
rome_neg_tibur_data.to_csv("rome_neg_tibur_data.csv")

In [10]:
# Export weekly
milan_cc_arese_data_w.to_csv("milan_cc_arese_data_w.csv")
milan_cc_busna_data_w.to_csv("milan_cc_busna_data_w.csv")
turin_cc_niche_data_w.to_csv("turin_cc_niche_data_w.csv")
turin_cc_beina_data_w.to_csv("turin_cc_beina_data_w.csv")
turin_cc_torin_data_w.to_csv("turin_cc_torin_data_w.csv")
turin_cc_todor_data_w.to_csv("turin_cc_todor_data_w.csv")
turin_cc_juve_data_w.to_csv("turin_cc_juve_data_w.csv")
rome_cc_eur2_data_w.to_csv("rome_cc_eur2_data_w.csv")
rome_neg_rmnaz_data_w.to_csv("rome_neg_rmnaz_data_w.csv")
rome_neg_tusc1_data_w.to_csv("rome_neg_tusc1_data_w.csv")
rome_neg_tusc2_data_w.to_csv("rome_neg_tusc2_data_w.csv")
rome_neg_tibur_data_w.to_csv("rome_neg_tibur_data_w.csv")